In [ ]:
%%writefile main.py

import streamlit as st
import home
import recall_by_year
import recall_by_maker
import faq

st.set_page_config(page_title="리콜 현황", layout="wide")

st.sidebar.header('📌 메뉴 선택')
menu = st.sidebar.radio('페이지를 선택하세요',['홈', '연도별 리콜 현황', '제조사별 리콜 현황', 'FAQ'])

if menu == '홈':
    home.show()
elif menu == '연도별 리콜 현황':
    recall_by_year.show()
elif menu == '제조사별 리콜 현황':
    recall_by_maker.show()
elif menu == 'FAQ':
    faq.show()


In [ ]:
%%writefile home.py

import streamlit as st

def show():
    st.markdown("""
        <style>
        .title {
            text-align: center;
            font-size: 60px;
            font-weight: bold;
            color: #2C3E50;
            margin-bottom: 20px;
        }
        .subtitle {
            text-align: center;
            font-size: 25px;
            color: #34495E;
            margin-bottom: 30px;
        }
        </style>
        <div class="title"> 🚘 RecallCheck 🚘</div>
        <div class="subtitle">🚗 자동차 리콜 현황 확인 및 FAQ 조회 시스템</div>
    """, unsafe_allow_html=True)

    col1, col2, col3 = st.columns([0.5, 3, 0.5]) 
    with col2:
        st.image("car_image2.jpg", use_container_width=True) 


In [ ]:
%%writefile recall_by_year.py

import streamlit as st
import pandas as pd
import mysql.connector
import plotly.express as px

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root1234',
    'port': 3306,
    'database': 'car_recall_db'
}

def get_recall_data():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # ✅ 기존 SQL 유지
        query = """
        SELECT recall_year, 
               SUM(kr_recall_car) AS kr_models, SUM(kr_recall_num) AS kr_units, 
               SUM(us_recall_car) AS us_models, SUM(us_recall_num) AS us_units, 
               SUM(total_recall_car) AS total_models, SUM(total_recall_num) AS total_units
        FROM car_recall_month
        GROUP BY recall_year
        ORDER BY recall_year DESC
        """
        cursor.execute(query)
        data = cursor.fetchall()
        
        # ✅ 데이터 프레임 생성
        df = pd.DataFrame(data, columns=['해당 연도', '국산 차종', '국산 대수', '수입 차종', '수입 대수', '총 차종', '총 대수'])

        # ✅ 🔥 **모든 값 1/2로 조정 (두 배로 나오는 문제 해결)**
        df.iloc[:, 1:] = df.iloc[:, 1:] / 2  

        # ✅ 총합 행 추가
        total_row = ['계'] + df.iloc[:, 1:].sum().tolist()
        df.loc[len(df)] = total_row

        # ✅ 다중 인덱스 헤더 설정
        df.columns = pd.MultiIndex.from_tuples([
            ("해당 연도", ""),
            ("국산자동차", "차종"), ("국산자동차", "대수"),
            ("수입자동차", "차종"), ("수입자동차", "대수"),
            ("계", "차종"), ("계", "대수")
        ])

        cursor.close()
        conn.close()

        return df

    except Exception as e:
        st.error(f"데이터를 불러오는 중 오류 발생: {e}")
        return pd.DataFrame()


def get_recall_graph_data():

    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # 차종 데이터 조회
        query_models = """
        SELECT recall_year, 
               SUM(kr_recall_car) AS kr_models, 
               SUM(us_recall_car) AS us_models
        FROM car_recall_month
        GROUP BY recall_year
        ORDER BY recall_year DESC
        """
        cursor.execute(query_models)
        data_models = cursor.fetchall()
        df_models = pd.DataFrame(data_models, columns=['해당 연도', '국산 리콜 차종', '수입 리콜 차종'])

        # 대수 데이터 조회
        query_units = """
        SELECT recall_year, 
               SUM(kr_recall_num) AS kr_units,
               SUM(us_recall_num) AS us_units
        FROM car_recall_month
        GROUP BY recall_year
        ORDER BY recall_year DESC
        """
        cursor.execute(query_units)
        data_units = cursor.fetchall()
        df_units = pd.DataFrame(data_units, columns=['해당 연도', '국산 리콜 대수', '수입 리콜 대수'])

        cursor.close()
        conn.close()

        return df_models, df_units

    except Exception as e:
        st.error(f"데이터를 불러오는 중 오류 발생: {e}")
        return pd.DataFrame(), pd.DataFrame()  


def show():
    st.title('📊 연도별 리콜 현황')

    df = get_recall_data()

    if df.empty:
        st.warning("📌 데이터가 없습니다. DB를 확인해주세요.")
        return

    st.markdown("""
        <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            text-align: center !important; /* 데이터 가운데 정렬 */
            border: 1px solid #ddd;
            padding: 10px;
            text-align: center;
        }
        th {
            background-color: #f4f4f4;
            font-weight: bold;
            text-align: center;
        }
        </style>
    """, unsafe_allow_html=True)

    st.markdown(df.to_html(index=False, escape=False), unsafe_allow_html=True)

    df_models, df_units = get_recall_graph_data()

    if df_models.empty or df_units.empty:
        st.warning("📌 그래프 데이터를 불러올 수 없습니다.")
        return

    df_models_melted = df_models.melt(id_vars=['해당 연도'], var_name="구분", value_name="리콜 차종 수")
    df_units_melted = df_units.melt(id_vars=['해당 연도'], var_name="구분", value_name="리콜 대수")

    # Plotly 그래프 생성 (차종 수)
    fig_models = px.bar(
        df_models_melted,
        x="해당 연도",
        y="리콜 차종 수",
        color="구분",
        barmode="group",
        title="연도별 리콜 차종 현황",
        color_discrete_map={"국산 리콜 차종": "pink", "수입 리콜 차종": "khaki"}
    )

    # Plotly 그래프 생성 (대수)
    fig_units = px.area(
        df_units_melted,
        x="해당 연도",
        y="리콜 대수",
        color="구분",
        title="연도별 리콜 대수 현황",
        color_discrete_map={"국산 리콜 대수": "lightblue", "수입 리콜 대수": "lightgreen"}
    )

    col1, col2 = st.columns(2)

    with col1:
        st.plotly_chart(fig_models, use_container_width=True)

    with col2:
        st.plotly_chart(fig_units, use_container_width=True)

In [ ]:
%%writefile recall_by_maker.py
import streamlit as st
import mysql.connector
import pandas as pd

# ✅ 데이터베이스 연결 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root1234',
    'port': 3306,
    'database': 'car_recall_db'
}

def get_recall_data():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        query = """
        SELECT car_model, SUM(car_num) AS total_recalls
        FROM car_recall_brand
        WHERE recall_month = '총합'
        GROUP BY car_model
        ORDER BY total_recalls DESC
        LIMIT 10;
        """
        cursor.execute(query)
        data = cursor.fetchall()
        
        df = pd.DataFrame(data, columns=['제조사', '리콜 건수'])
        df.index = df.index + 1

        cursor.close()
        conn.close()

        return df

    except Exception as e:
        st.error(f"데이터를 불러오는 중 오류 발생: {e}")
        return pd.DataFrame()

def show():
    st.title("🚗 차량 제조사별 리콜 현황 (TOP 10)")
    st.markdown(
        "<p style='text-align: right; font-size: 15px; color: gray;'>기준: 2022-2024</p>",
        unsafe_allow_html=True
    )

    df = get_recall_data()

    if df.empty:
        st.warning("📌 데이터가 없습니다.")
        return
]    st.markdown(
        """
        <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th {
            background-color: #f0f0f0 !important;
            text-align: center !important;
            font-weight: bold;
            padding: 10px;
        }
        td {
            text-align: center !important;
            padding: 10px;
            border-bottom: 1px solid #ddd;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    # ✅ HTML 테이블 렌더링
    df_html = df.to_html(escape=False, index=True, classes='dataframe')
    st.markdown(df_html, unsafe_allow_html=True)


In [ ]:
%%writefile faq.py
import streamlit as st
import mysql.connector
import pandas as pd

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root1234',
    'port': 3306,
    'database': 'car_recall_db'
}

def get_faq_data(brand, keyword=""):
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        if brand == "기아":
            query = "SELECT kia_question, kia_answer FROM kia_faq"
        else:  # 현대
            query = "SELECT hyundai_question, hyundai_answer FROM hyundai_faq"
        
        cursor.execute(query)
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=['질문', '답변'])

        cursor.close()
        conn.close()

        # 검색 기능
        if keyword.strip():
            df = df[df["질문"].str.contains(keyword, case=False, na=False)]  

        return df

    except Exception as e:
        st.error(f"데이터를 불러오는 중 오류 발생: {e}")
        return pd.DataFrame() 


def show():
    """Streamlit FAQ 검색 & 선택 기능 페이지"""
    st.title("🚗 브랜드별 FAQ")

    brand = st.selectbox("브랜드를 선택하세요", ["기아", "현대"])

    keyword = st.text_input("검색할 키워드를 입력하세요", "")

    df = get_faq_data(brand, keyword)

    if df.empty:
        st.warning("📌 검색 결과가 없습니다.")
        return

    question = st.selectbox("질문을 선택하세요", df["질문"].tolist())
    answer = df[df["질문"] == question]["답변"].iloc[0].replace("\n", " ").strip()
    st.write(f"**답변:** {answer}")
